In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df_links = pd.read_csv('csv/links.csv')

In [3]:
df_movies = pd.read_csv('csv/movies.csv')

In [4]:
df_ratings = pd.read_csv('csv/ratings.csv')

In [5]:
df_tags = pd.read_csv('csv/tags.csv')

In [6]:
header = {
    'User-Agent': 'Chrome 108.0.5359.125',
    'Accept-Language': 'es'
}

In [7]:
columnaSinopsis=[]
#for idPeli in df_links['tmdbId'].head(10):
for idPeli in df_links['tmdbId']:
    print(int(idPeli))
    try:
        url = "https://www.themoviedb.org/movie/"+str(int(idPeli))
        page = requests.get(url,headers=header)
        soup = BeautifulSoup (page.content, 'html.parser')
        sinopsis = soup.find(class_="overview")
        sinopsis = sinopsis.text
        sinopsis = str(sinopsis)
        #print(sinopsis)
        columnaSinopsis.append(sinopsis)
    except:
        columnaSinopsis.append("Sin Informacion")
columnaSinopsis

862
8844
15602
31357
11862
949
11860
45325
9091
710


['\nLos juguetes de Andy, un niño de seis años, temen que haya llegado su hora y que un nuevo regalo de cumpleaños les sustituya en el corazón de su dueño. Woody, un vaquero que ha sido hasta ahora el juguete favorito de Andy, trata de tranquilizarlos hasta que aparece Buzz Lightyear, un héroe espacial dotado de todo tipo de avances tecnológicos. Woody es relegado a un segundo plano., pero su constante rivalidad se transformará en una gran amistad cuando ambos se pierden en la ciudad sin saber cómo volver a casa.\n',
 '\nAlan Parris queda atrapado en un juego de mesa mágico y muy antiguo, Jumanji, durante 25 años. Cuando finalmente es liberado por dos niños, una manada de animales exóticos también es liberada. Alan deberá salvar a su pueblo de la destrucción.\n',
 '\nEl verano ha llegado a Minnesotta y la relación entre Max y John ha mejorado considerablemente, tanto que se han hecho amigos. Además, comparten una gran afición por la pesca y pasan largas horas en el lago. María, una atr